In [22]:
from fastapi import FastAPI
import pandas as pd
import numpy as np
from transformers import pipeline
from sklearn.cluster import KMeans
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#import nltk
import folium
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from IPython.display import display
from typing import List

In [5]:
# Cargamos el DataFrame desde el archivo pickle
df = pd.read_pickle('D:/GCP/API-Proy/data/ML_Dataset_filtrado.pickle')

In [7]:
# Función para determinar el estado basado en latitud y longitud
def determinar_estado(lat, lon):
    if 32.5 <= lat <= 42 and -124 <= lon <= -114:
        return 'California'
    elif 24.5 <= lat <= 31 and -87.5 <= lon <= -80:
        return 'Florida'
    elif 39 <= lat <= 42 and -80.5 <= lon <= -74.5:
        return 'Pennsylvania'
    elif 35 <= lat <= 36.7 and -90 <= lon <= -81.5:
        return 'Tennessee'
    elif 25.8 <= lat <= 36.5 and -106.5 <= lon <= -93.5:
        return 'Texas'
    elif 40.5 <= lat <= 45 and -79.75 <= lon <= -71.75:
        return 'New York'
    else:
        return 'Unknown'


# Eliminar la columna 'city'
df.drop(columns=['city'], inplace=True)

# Asignar el estado correcto basado en latitud y longitud
df['state'] = df.apply(lambda row: determinar_estado(row['latitude'], row['longitude']), axis=1)

# Filtrar el dataset para mantener solo los registros con estados conocidos
df = df[df['state'] != 'Unknown']

# Mostrar el DataFrame actualizado y un resumen de los estados
df[['latitude', 'longitude', 'state']].head()
print(df['state'].value_counts())

     latitude   longitude         state
0   39.949334  -75.166176  Pennsylvania
10  34.421239 -119.641027    California
12  27.945201  -82.527294       Florida
16  27.925290  -82.515989       Florida
19  34.421239 -119.641027    California
state
Pennsylvania    268631
Florida         256560
California      177814
Texas           137642
New York        109761
Tennessee       105105
Name: count, dtype: int64


#### Modelo

In [15]:
# Filtrar los datos por estado y tipo de comida
# En el deploy cambiará por el Estado y la selección de tipo de comida del usuario
estado_seleccionado = 'Florida'
tipo_comida_seleccionado = 'Italiana'

df_filtrado = df[(df['state'] == estado_seleccionado) & (df['category_name'] == tipo_comida_seleccionado)]

In [16]:
# Calcular densidad de restaurantes
def calcular_densidad(latitudes, longitudes, radio=1.0):
    coords = np.array(list(zip(latitudes, longitudes)))
    nbrs = NearestNeighbors(radius=radio, metric='haversine').fit(np.radians(coords))
    densidad = nbrs.radius_neighbors_graph(np.radians(coords)).sum(axis=1)
    return densidad

df_filtrado['densidad_restaurantes'] = calcular_densidad(df_filtrado['latitude'], df_filtrado['longitude'])

# Calcular distancia a restaurantes populares
restaurantes_populares = df[df['business_stars'] >= 4][['latitude', 'longitude']]
coords_populares = np.array(list(zip(restaurantes_populares['latitude'], restaurantes_populares['longitude'])))
nbrs_populares = NearestNeighbors(n_neighbors=1, metric='haversine').fit(np.radians(coords_populares))
distancias, _ = nbrs_populares.kneighbors(np.radians(df_filtrado[['latitude', 'longitude']]))
df_filtrado['distancia_restaurantes_populares'] = distancias * 6371  # Convertir a kilómetros

C:\Users\Familia Velazco\AppData\Local\Temp\ipykernel_1132\4260237364.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['densidad_restaurantes'] = calcular_densidad(df_filtrado['latitude'], df_filtrado['longitude'])
d:\GCP\.venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
C:\Users\Familia Velazco\AppData\Local\Temp\ipykernel_1132\4260237364.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df

In [10]:
df_filtrado.head(3)

,business_id,state,latitude,longitude,business_stars,business_platform,review_stars,review_platform,category_name,sentimiento,densidad_restaurantes,distancia_restaurantes_populares
851,0x808e30f8a00bc9bb:0x2259e3faa7b24b9e,California,37.821445,-121.999198,4.3,1,5,1,Italiana,5,21055.0,0.000000
3469,0x80dd304bb43ca2f7:0x9e38f711f2e4a30e,California,33.761795,-118.138818,4.4,1,5,1,Italiana,5,21055.0,0.000000
3556,0x80843e07e92c87b3:0x1cbd1bb3eebae67,California,38.549591,-122.806446,3.6,1,4,1,Italiana,5,21055.0,0.494542


In [17]:
# Seleccionar las columnas necesarias, excluyendo latitude y longitude de la normalización
columnas_para_escalar = [
    'business_stars', 'business_platform', 'review_stars', 'review_platform', 
    'sentimiento', 'densidad_restaurantes', 'distancia_restaurantes_populares'
]

# Escalado de características numéricas (sin incluir latitude y longitude)
scaler = StandardScaler()
df_filtrado[columnas_para_escalar] = scaler.fit_transform(df_filtrado[columnas_para_escalar])

C:\Users\Familia Velazco\AppData\Local\Temp\ipykernel_1132\11948661.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado[columnas_para_escalar] = scaler.fit_transform(df_filtrado[columnas_para_escalar])


In [18]:
# Aplicar KMeans clustering para encontrar clusters de ubicaciones positivas, sin escalar latitud y longitud
locations = df_filtrado[['latitude', 'longitude']]
otras_caracteristicas = df_filtrado[columnas_para_escalar]

# Concatenar las características sin escalar latitud y longitud
features = pd.concat([locations.reset_index(drop=True), otras_caracteristicas.reset_index(drop=True)], axis=1)

# Parámetro para personalizar el número de centroides
num_centroides = 5  # Este valor se puede cambiar según las necesidades

kmeans = KMeans(n_clusters=num_centroides, random_state=42)
kmeans.fit(features)

# Obtener los centroides de los clusters como las ubicaciones recomendadas
centroides = kmeans.cluster_centers_

In [36]:
centroides

array([[ 2.79653749e+01, -8.25868984e+01,  1.85573121e-03,
         7.05013650e-01,  3.15442826e-01,  6.92058422e-01,
         5.65449144e-01,  0.00000000e+00, -1.06695458e-01],
       [ 2.85025972e+01, -8.19581193e+01,  6.18104143e-01,
        -1.41432753e+00,  3.07419661e-01, -1.38908022e+00,
        -1.75349407e-01,  0.00000000e+00, -9.85725132e-02],
       [ 2.79931288e+01, -8.25604173e+01, -1.21587464e+00,
         6.76537547e-01, -1.70599554e+00,  6.76429291e-01,
        -1.46200024e+00,  0.00000000e+00,  3.54836872e-02],
       [ 2.62628301e+01, -8.05326988e+01,  5.42597891e-01,
        -1.41432753e+00,  2.54393502e-01, -1.39553925e+00,
        -2.67477114e-01,  0.00000000e+00, -1.57482374e-01],
       [ 2.93203231e+01, -8.25490641e+01, -5.58679442e-01,
        -1.41432753e+00, -2.49328240e-01, -1.42408175e+00,
        -7.01341576e-01,  0.00000000e+00,  8.44013755e+00]])

In [19]:
# Diccionario para centrar el mapa en los estados seleccionados
centro_estados = {
    'California': [36.7783, -119.4179],
    'Florida': [27.9944024, -81.7602544],
    'Pennsylvania': [41.2033, -77.1945],
    'Tennessee': [35.5175, -86.5804],
    'Texas': [31.9686, -99.9018],
    'New York': [40.7128, -74.0060]
}

In [20]:
# Crear un mapa centrado en el estado seleccionado
centro_estado = centro_estados[estado_seleccionado]
mapa = folium.Map(location=centro_estado, zoom_start=7)

# Agregar solo los centroides (ubicaciones recomendadas) al mapa con latitud y longitud en el popup
for centroide in centroides:
    folium.Marker(
        location=[centroide[0], centroide[1]],
        popup=f'Ubicación Recomendada<br>Latitud: {centroide[0]}<br>Longitud: {centroide[1]}',
        icon=folium.Icon(color='red', icon='star')
    ).add_to(mapa)

# Mostrar el mapa en el notebook
display(mapa)

In [1]:
import requests
import numpy as np

def obtener_datos(estado: str, tipo_de_comida: str):
    url = f"http://127.0.0.1:8000/recomendacion/{estado},{tipo_de_comida}"  # Asegúrate de que esta URL sea la correcta para tu API
    #params = {"estado": estado, "tipo_de_comida": tipo_de_comida}
    
    response = requests.get(url)
    
    if response.status_code == 200:
        datos = response.json()
        # Convertir los datos a un arreglo de NumPy
        arreglo = np.array(datos)
        return arreglo
    else:
        response.raise_for_status()

# Llamada de ejemplo
estado = "Tennessee"
tipo_de_comida = "Italiana"
resultado = obtener_datos(estado, tipo_de_comida)

print(type(resultado))  # Verifica que el tipo sea numpy.ndarray
resultado


<class 'numpy.ndarray'>


array([[ 3.61194787e+01, -8.67625246e+01, -8.77312102e-02,
         7.51309090e-01,  2.37027947e-01,  7.47472353e-01,
         5.73710017e-01,  0.00000000e+00, -1.59041764e-01],
       [ 3.60581089e+01, -8.35589354e+01,  7.84751639e-01,
        -1.32870274e+00,  3.72192939e-01, -1.31923302e+00,
        -2.17777550e-01,  0.00000000e+00,  5.86549801e-02],
       [ 3.61340090e+01, -8.67691076e+01, -1.31261402e+00,
         7.02633629e-01, -1.72093385e+00,  6.98588480e-01,
        -1.59472454e+00,  0.00000000e+00, -7.65156671e-02],
       [ 3.53140529e+01, -8.96670678e+01,  4.13576038e-01,
        -1.32870274e+00,  2.37900561e-01, -1.30441023e+00,
        -2.12303108e-01,  0.00000000e+00,  7.69360435e-01],
       [ 3.58750036e+01, -8.66705815e+01,  5.42155217e-01,
        -1.32870274e+00,  3.14559910e-01, -1.32804732e+00,
        -1.64802681e-01,  0.00000000e+00,  3.06727478e-01]])

In [37]:
resultado

array([[ 2.79653749e+01, -8.25868984e+01,  1.85573121e-03,
         7.05013650e-01,  3.15442826e-01,  6.92058422e-01,
         5.65449144e-01,  0.00000000e+00, -1.06695458e-01],
       [ 2.85025972e+01, -8.19581193e+01,  6.18104143e-01,
        -1.41432753e+00,  3.07419661e-01, -1.38908022e+00,
        -1.75349407e-01,  0.00000000e+00, -9.85725132e-02],
       [ 2.79931288e+01, -8.25604173e+01, -1.21587464e+00,
         6.76537547e-01, -1.70599554e+00,  6.76429291e-01,
        -1.46200024e+00,  0.00000000e+00,  3.54836872e-02],
       [ 2.62628301e+01, -8.05326988e+01,  5.42597891e-01,
        -1.41432753e+00,  2.54393502e-01, -1.39553925e+00,
        -2.67477114e-01,  0.00000000e+00, -1.57482374e-01],
       [ 2.93203231e+01, -8.25490641e+01, -5.58679442e-01,
        -1.41432753e+00, -2.49328240e-01, -1.42408175e+00,
        -7.01341576e-01,  0.00000000e+00,  8.44013755e+00]])